# Imports

In [ ]:
from pathlib import Path
import re

from more_itertools import powerset

import numpy as np
import pandas as pd

import statsmodels.api as sm
from statsmodels.formula.api import ols

from constants import DataSplit, Model, Metric, DATASET_SYMBOLS, CONSENSUS_BASELINES

# Script Parameters

In [ ]:
data_dir = r'./out'  # Directory containing experiment data to analyze
concat_results = True  # If True, join all files within data_dir (and subdirectories, used with replications). If False, only use <data_dir>/results.csv
alpha = 0.05  # Desired alpha for statistical testing (threshold for ANOVA model reduction)

# Print settings
only_reduced = True  # Only display/print reduced models
latex_output = False  # Print LaTeX instead of DataFrames
combine_anova_latex = True  # Combine full and reduced ANOVA models into one LaTeX table (REQUIRES latex_output = True, OVERRIDES only_reduced)

# Function Definitions

In [ ]:
# Pivot table aggregation technique
pivot_agg_func = (lambda x: f'${np.mean(x):.3f} \\pm {np.std(x):.3f}$') if latex_output else [np.mean, np.std]

# These functions are used to clean up the ANOVA and LaTeX printouts
def clean_effect_name(name):
    return re.findall(r'"([^"]*)"', name)[0][0] if '"' in name else name

def fix_ordering(df, level=1):
    return df.reindex(columns=df.columns.reindex([metric.value for metric in Metric], level=level)[0])

def get_style(df):
    return df.style.set_table_styles([
        {'selector': 'toprule', 'props': ':hline;'},
        {'selector': 'midrule', 'props': ':hline\hline;'},
        {'selector': 'bottomrule', 'props': ':hline;'}
    ])

def get_anova_latex(df, name):
    df = df.loc[:, (slice(None), ['Coefficient','PR(>F)'])]
    df = df.rename(index={'Residual/Intercept': 'Intercept'}, columns={'PR(>F)': 'P-value'})
    df = fix_ordering(df, level=0)

    s = get_style(df)
    s.format({(metric, col): f'${{:.{precision}f}}$'
              for metric in Metric
              for col, precision in {'Coefficient':5, 'P-value':3}.items()})
    if combine_anova_latex:
        s.format_index('\\rotatebox[origin=c]{{90}}{{{}}}', level=0)

    col_format = '|c|'*combine_anova_latex + '|l|' + 'rc|'*(len(df.columns)//2)

    latex = s.to_latex(column_format=col_format, caption=name, multicol_align='|c|', position='htbp', position_float='centering')
    latex = latex.replace('$nan$','-') # replace NaNs with dashes
    latex = latex.replace('\\\\\nIntercept', '\\\\ \hdashline\nIntercept') # put hdashline before intercept row
    latex = latex.replace('\hline \hdashline', '\hline') # remove hdashline if placed after hline (occurs when intercept is right under the toprule
    if combine_anova_latex:
        latex = latex.replace('\\\\\n &  &', '\\\\\nModel & Effect &') # index names (letting pandas handle this gets the header wrong)
        latex = latex.replace('\\\\\n\multirow', '\\\\ \hline\hline\n\multirow') # put double hline above reduced model
    else:
        latex = latex.replace('\n & Coefficient', '\nEffect & Coefficient') # index name (letting pandas handle this gets the header wrong)

    return latex

# Data Loading

In [ ]:
path = Path(data_dir)

pattern = rf'{"**/*" if concat_results else ""}results.csv'

data = pd.concat([pd.read_csv(filename, index_col=0) for filename in path.glob(pattern)], ignore_index=True)
display(data)

# Asset Presence Analysis (model-averaged, baseline models and training data excluded)

In [ ]:
df = data.loc[data[DataSplit.TEST] & ~data[Model.RANDOM_BASELINE] & ~data[Model.CONSTANT_BASELINE] & ~data[Model.PREVIOUS_BASELINE] & ~data[Model.CONSENSUS_BASELINE]]

pivot = [
    df.loc[~df['Random']]
      .pivot_table(values=[metric for metric in Metric],
                   index=asset_type,
                   aggfunc=pivot_agg_func)
    for asset_type in DATASET_SYMBOLS.keys()
]

pivot += [
    df[~df[[asset_type for asset_type in DATASET_SYMBOLS.keys()] + ['Random']].any(axis=1)]
      .pivot_table(values=[metric for metric in Metric],
                   index='SPY',
                   aggfunc=pivot_agg_func),
    df.loc[df['Random']]
      .pivot_table(values=[metric for metric in Metric],
                   index='Random',
                   aggfunc=pivot_agg_func)
]

pivot = pd.concat(pivot, keys=[tab.index.name for tab in pivot], names=['Asset Type','Presence']) \
          .rename(index={'SPY': 'SPY-Only', 'Random': 'Random Data'}) \
          .reindex([True, False], level=1)

if latex_output:
    print(get_style(pivot).to_latex(column_format='|lc|ccc|', position='htbp', position_float='centering'))
else:
    display(fix_ordering(pivot))

# Asset Combinations (model-averaged, baseline models and training data excluded)

In [ ]:
df = data.loc[data[DataSplit.TEST] & ~data[Model.RANDOM_BASELINE] & ~data[Model.CONSTANT_BASELINE] & ~data[Model.PREVIOUS_BASELINE] & ~data[Model.CONSENSUS_BASELINE]].copy()

for asset_type in DATASET_SYMBOLS.keys():
    df[asset_type] = df[asset_type].map({True: asset_type[0], False: ''})

df['Random'] = df['Random'].map({True: 'Random Data', False: ''})

df['Asset Combination'] = df[[asset_type for asset_type in DATASET_SYMBOLS.keys()] + ['Random']].apply(lambda x: ''.join(x.values.astype(str)), axis=1)

pivot = df.pivot_table(values=[metric for metric in Metric],
                       index='Asset Combination',
                       aggfunc=pivot_agg_func)

pivot = pivot.rename(index={'': 'SPY-Only'})
pivot = pivot.reindex([''.join(c) for c in powerset(''.join(asset_type[0] for asset_type in DATASET_SYMBOLS.keys()))] + ['SPY-Only', 'Random Data'])
pivot = pivot.drop(index='')

if latex_output:
    print(get_style(pivot).to_latex(column_format='|l|ccc|', position='htbp', position_float='centering'))
else:
    display(fix_ordering(pivot))

# Model Performance (dataset averaged, random data excluded)
## Out-sample

In [ ]:
df = data.loc[~data['Random']]

pivot = [
    df.loc[df[DataSplit.TEST]]
      .pivot_table(values=[metric for metric in Metric],
                   index=model,
                   aggfunc=pivot_agg_func)
    for model in Model
]

pivot = pd.concat(pivot, keys=[tab.index.name for tab in pivot], names=['Model'])
pivot = pivot.loc[pivot.index.get_level_values(1)].droplevel(1)

if latex_output:
    print(get_style(pivot).to_latex(column_format='|l|ccc|', position='htbp', position_float='centering'))
else:
    display(fix_ordering(pivot))

## In-sample

In [ ]:
df = data.loc[~data['Random']]

pivot = [
    df.loc[~df[DataSplit.TEST]]
      .pivot_table(values=[metric for metric in Metric],
                   index=model,
                   aggfunc=pivot_agg_func)
    for model in Model
]

pivot = pd.concat(pivot, keys=[tab.index.name for tab in pivot], names=['Model','used'])
pivot = pivot.loc[pivot.index.get_level_values(1)].droplevel(1)

if latex_output:
    print(get_style(pivot).to_latex(column_format='|l|ccc|', position='htbp', position_float='centering'))
else:
    display(fix_ordering(pivot))

# Statistical Analysis of Factor Effects (random data baseline excluded)
## ANOVA Model Generation

In [ ]:
all_df = data.loc[data[DataSplit.TEST] & ~data['Random']]
all_df = all_df.replace({True: 1, False: -1}) # required to get coefficients. !does not change results!

anovas = {}

for model in Model:
    df = all_df.loc[data[model]]

    anovas[model] = {
        'full': {},
        'reduced': {}
    }

    if not latex_output:
        print(f'\nAnalyzing {model}...')

    for metric in Metric:
        if model in CONSENSUS_BASELINES and metric is Metric.ROC_AUC:
            continue

        main_effects = ['Q("'+asset_type+'")' for asset_type in DATASET_SYMBOLS.keys()]

        relation = f'Q("{metric}") ~ ' + ' + '.join(main_effects)
        glm = ols(relation, data=df).fit()
        aov = sm.stats.anova_lm(glm, typ=1)

        aov = aov.rename(index={'Residual': 'Residual/Intercept'})
        coefs = glm.params.rename(index={'Intercept': 'Residual/Intercept'})
        coefs.name = 'Coefficient'

        anovas[model]['full'][metric.value] = aov.join(coefs)

        if not latex_output:
            print(f'\nReducing over {metric}...')

        no_reductions = True

        # while non-significant effects, remove the least significant effect and associated interactions and refit ANOVA model
        while (aov['PR(>F)'] > alpha).any():
            no_reductions = False
            rem_effect = aov['F'].idxmin()
            if not latex_output:
                print(f'removing effect {clean_effect_name(rem_effect)} (p={aov["PR(>F)"].max():.3f})')

            main_effects.remove(rem_effect)

            relation = f'Q("{metric}") ~ '
            relation += ' + '.join(main_effects) if len(main_effects) > 0 else '1'

            glm = ols(relation, data=df).fit()
            aov = sm.stats.anova_lm(glm, typ=1)

        if no_reductions:
            if not latex_output:
                print('no effects removed')
        else:
            coefs = glm.params.rename(index={'Intercept': 'Residual/Intercept'})
            coefs.name = 'Coefficient'
            aov = aov.rename(index={'Residual': 'Residual/Intercept'})
            anovas[model]['reduced'][metric.value] = aov.join(coefs)

## Printouts

In [ ]:
index_ordering = [asset_type[0] for asset_type in DATASET_SYMBOLS.keys()] + ['Residual/Intercept']

for model, to_join in anovas.items():
    full_join = to_join['full']
    reduced_join = to_join['reduced']

    full_model = pd.concat(full_join, axis=1)
    full_model = full_model.set_index(full_model.index.map(clean_effect_name)).reindex(index_ordering)

    reduced_model = None

    if len(reduced_join) == len(Metric):
        reduced_model = pd.concat(reduced_join, axis=1)
    elif len(reduced_join) > 0:
        to_join = {metric: reduced_join[metric]
        if metric in reduced_join.keys()
        else full_join[metric]
                   for metric in full_join.keys()}
        reduced_model = pd.concat(to_join, axis=1)

    if len(reduced_join) == 0: # no reduction occurred
        if latex_output: # print LaTeX
            if combine_anova_latex: # combined table format
                print(get_anova_latex(pd.concat({'Full \& Reduced': full_model}), f'{model}'))
            else: # individual table format
                print(get_anova_latex(full_model, f'{model} (Full \& Reduced)'))
        else: # display DataFrame
            print(f'\n{model} (Full & Reduced):')
            display(full_model)
    else:  # reduction occurred
        reduced_model = reduced_model.set_index(reduced_model.index.map(clean_effect_name)).reindex(index_ordering)
        reduced_model = reduced_model.dropna(how='all')
        if latex_output:  # print LaTeX
            if combine_anova_latex: # combine full and reduced ANOVA models into one table
                print(get_anova_latex(pd.concat({'Full':full_model, 'Reduced':reduced_model}), f'{model}'))
            else:  # individual table format
                if not only_reduced:  # print full model if desired
                    print(get_anova_latex(full_model, f'{model} (Full)'))
                print(get_anova_latex(reduced_model, f'{model} (Reduced)'))
        else:  # display DataFrames
            if not only_reduced:
                print(f'\n{model} (Full):')
                display(full_model)
            print(f'\n{model} (Reduced):')
            display(reduced_model)